In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file_names = [
    'trhs201652.dat',
    'trhs201752.dat',
    'trhs201852.dat',
    'trhs201952.dat',
    'trhs202052.dat'
]

data_directory = '/Users/bercinersoz/Desktop/data 2/flows'

# All files into a single DataFrame
facttable_df = pd.concat([pd.read_csv(f"{data_directory}/{file_name}", delimiter=',') for file_name in file_names], ignore_index=True)
facttable_df['PERIOD'] = facttable_df['PERIOD'].astype(str).str[:4]

print(facttable_df.head())

In [ ]:
print(facttable_df.tail())

In [ ]:
# Display general information about the dataframe
print("DataFrame Information:")
print(facttable_df.info())

# Display summary statistics of the dataframe
print("\nSummary Statistics:")
print(facttable_df.describe(include='all'))

# Display the number of missing values for each column
print("\nMissing Values:")
print(facttable_df.isnull().sum())

# Display the data types of each column
print("\nData Types:")
print(facttable_df.dtypes)

In [ ]:

reporters_file = '/Users/bercinersoz/Desktop/data 2/metadata/REPORTERS.txt'
partners_file = '/Users/bercinersoz/Desktop/data 2/metadata/PARTNERS.txt'
cn_modified_file = '/Users/bercinersoz/Desktop/data 2/metadata/CN_modified.txt'



In [ ]:
cn_modified_df = pd.read_csv(cn_modified_file, delimiter=',')
cn_modified_df['HS2']=cn_modified_df['HS2'].astype(str).str.zfill(2)  #problem solved in HS2 0 to 00
print("CN DataFrame:")
print(cn_modified_df.head())

In [ ]:
keywords = ['orange', 'tangerine', 'lemon', 'grapefruit', 'citrus']

# Convert HS6_NAME to lowercase for case-insensitive search
cn_modified_df['HS6_NAME'] = cn_modified_df['HS6_NAME'].str.lower()

# Find rows where HS6_NAME contains any of the keywords
citrus_products_df = cn_modified_df[cn_modified_df['HS6_NAME'].str.contains('|'.join(keywords))]

# Extract the relevant columns
citrus_hs6_codes = citrus_products_df[['HS6', 'HS6_NAME', 'HS4_NAME', 'HS2_NAME']]

citrus_hs6_codes.to_csv('/Users/bercinersoz/Desktop/citrus_products_filtered.csv', index=False)

# HS6 codes to exclude
exclude_codes = ['081090', '081340', '200799', '200980', '200989', '282420', '330129']

# Filter out
citrus_hs6_codes = citrus_hs6_codes[~citrus_hs6_codes['HS6'].isin(exclude_codes)]

print("Citrus Products and their HS6 Codes:")
print(citrus_hs6_codes)




In [ ]:
# Filter facttable_df, without copy gives warning!
filtered_facttable_df = facttable_df[facttable_df['PRODUCT_HS'].isin(citrus_hs6_codes['HS6'])].copy()

# Ensure Product HS is a string
filtered_facttable_df['PRODUCT_HS'] = filtered_facttable_df['PRODUCT_HS'].astype(str)
cn_modified_df['HS6'] = cn_modified_df['HS6'].astype(str)

# Perform a left join on PRODUCT_HS
filtered_facttable_df = filtered_facttable_df.merge(cn_modified_df[['HS6', 'HS4_NAME']], left_on='PRODUCT_HS', right_on='HS6', how='left')

unique_hs4_names = filtered_facttable_df['HS4_NAME'].unique()

print("Unique HS4_NAME values:")
print(unique_hs4_names)

# Truncate HS4_NAME to the first 30 characters
filtered_facttable_df.loc[:, 'HS4_NAME'] = filtered_facttable_df['HS4_NAME'].str[:30]

# Drop the HS6 column
filtered_facttable_df = filtered_facttable_df.drop(columns=['HS6'])

print(filtered_facttable_df.head())

filtered_facttable_df.info()

In [ ]:
reporters_df = pd.read_csv(reporters_file, delimiter="\t", names=["ID", "Date_since_declarent", "Date_until_declarent", "Name", "Date_since_declarent2", "Date_until_declarent2"], encoding='ISO-8859-1')
reporters_df = reporters_df.iloc[:, 0:4]
reporters_df["ID"] = reporters_df["ID"].str.lstrip('0')
reporters_df["Name"] = reporters_df["Name"].str.strip()

partners_df = pd.read_csv(partners_file, delimiter="\t", names=["ID", "Date_since_partner", "Date_until_partner", "Name", "Date_since_partner2", "Date_until_partner2"], encoding='ISO-8859-1')
partners_df = partners_df.iloc[:, 0:4]
partners_df["ID"] = partners_df["ID"].str.lstrip('0')
partners_df["Name"] = partners_df["Name"].str.strip()

# Exist (unique, between 31.12.2015-31.12.2020)
reporters_exits = reporters_df.groupby('Name').filter(lambda x: len(x) == 1 and '31/12/2015' < x['Date_until_declarent'].iloc[0] <= '31/12/2020')
partners_exits = partners_df.groupby('Name').filter(lambda x: len(x) == 1 and '31/12/2015' < x['Date_until_partner'].iloc[0] <= '31/12/2020')

# Entrance (unique, between 31.12.2015-31.12.2020)
reporters_entries = reporters_df.groupby('Name').filter(lambda x: len(x) == 1 and x['Date_since_declarent'].iloc[0] > '31/12/2015')
partners_entries = partners_df.groupby('Name').filter(lambda x: len(x) == 1 and x['Date_since_partner'].iloc[0] > '31/12/2015')

reporters_exits_countries = reporters_exits[['Name', 'Date_until_declarent']].drop_duplicates()
partners_exits_countries = partners_exits[['Name', 'Date_until_partner']].drop_duplicates()

reporters_entries_countries = reporters_entries[['Name', 'Date_since_declarent']].drop_duplicates()
partners_entries_countries = partners_entries[['Name', 'Date_since_partner']].drop_duplicates()

print("Reporters Exits Countries (31.12.2015 - 31.12.2020):")
print(reporters_exits_countries)

print("Partners Exits Countries (31.12.2015 - 31.12.2020):")
print(partners_exits_countries)

print("Reporters Entries Countries (After 31.12.2015):")
print(reporters_entries_countries)

print("Partners Entries Countries (After 31.12.2015):")
print(partners_entries_countries)

In [ ]:
# Filter out GB from declarant countries
filtered_facttable_df = filtered_facttable_df[filtered_facttable_df['DECLARANT_ISO'] != 'GB']


In [ ]:
world_bank_gdp_file = '/Users/bercinersoz/Desktop/data 2/worldbank_gdp.csv'

# Read the World Bank GDP data and select only the specified columns
columns_to_keep = ['Country Name', 'Country Code', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]', '2019 [YR2019]']
world_bank_gdp_df = pd.read_csv(world_bank_gdp_file, usecols=columns_to_keep, encoding='ISO-8859-1')

world_bank_gdp_df.columns = ['Country Name', 'Country Code', '2016', '2017', '2018', '2019']

world_bank_gdp_df['Country Code'] = world_bank_gdp_df['Country Code'].str.strip()  #problem solved to unmatch case

# Convert GDP columns to numeric (float)
for year in ['2016', '2017', '2018', '2019']:
    world_bank_gdp_df[year] = pd.to_numeric(world_bank_gdp_df[year], errors='coerce')

# Define the USD to Euro conversion rates for the respective years based on ECB data
usd_to_euro_rates = {
    '2016': 0.9048,
    '2017': 0.8852,
    '2018': 0.8471,
    '2019': 0.8933
}

# Convert GDP values from USD to Euro
for year in ['2016', '2017', '2018', '2019']:
    world_bank_gdp_df[year] = world_bank_gdp_df[year] * usd_to_euro_rates[year]

# Melt the DataFrame to long format
melted_gdp_df = pd.melt(world_bank_gdp_df, id_vars=['Country Name', 'Country Code'], var_name='Year', value_name='GDP')

print("Data types after conversion:")
print(melted_gdp_df.dtypes)

print("Melted World Bank GDP DataFrame (in Euro):")
print(melted_gdp_df.head())



In [ ]:
melted_gdp_df.info()

In [ ]:
transport_mode_data = {
    'ID': [0, 1, 2, 3, 4, 5, 7, 8, 9],
    'TRANSPORTATION': [
        'Unknown', 'Sea', 'Rail', 'Road', 'Air', 'Post', 
        'Fixed Mechanism', 'Inland Waterway', 'Self Propulsion'
    ]
}

# Create a DataFrame from the transport mode data
transport_mode_df = pd.DataFrame(transport_mode_data)

In [ ]:
flow_data = {
    'ID': [1, 2],
    'TRADE_FLOW': ['Import', 'Export']
}

# Create a DataFrame from the flow data
flow_df = pd.DataFrame(flow_data)

In [ ]:
print(flow_df)

In [ ]:
iso_codes_file = '/Users/bercinersoz/Desktop/data 2/metadata/iso_codes.csv'

iso_codes_df = pd.read_csv(iso_codes_file, delimiter=";", usecols=['iso2', 'iso3'])

print("ISO Codes DataFrame:")
print(iso_codes_df.head())

In [ ]:
iso2_to_iso3 = iso_codes_df.set_index('iso2')['iso3'].to_dict()

# Ensure the key columns are of the same type and correct any potential issues with column names
filtered_facttable_df.loc[:, 'DECLARANT_ISO'] = filtered_facttable_df['DECLARANT_ISO'].astype(str)
filtered_facttable_df.loc[:, 'PARTNER_ISO'] = filtered_facttable_df['PARTNER_ISO'].astype(str)

# Map DECLARANT_ISO and PARTNER_ISO to their 3-digit equivalents using the mapping dictionary
filtered_facttable_df.loc[:, 'DECLARANT_ISO_3'] = filtered_facttable_df.loc[:, 'DECLARANT_ISO'].map(iso2_to_iso3)
filtered_facttable_df.loc[:, 'PARTNER_ISO_3'] = filtered_facttable_df.loc[:, 'PARTNER_ISO'].map(iso2_to_iso3)

# Drop the original DECLARANT_ISO and PARTNER_ISO columns
filtered_facttable_df = filtered_facttable_df.drop(columns=['DECLARANT_ISO', 'PARTNER_ISO'])

print("Updated Filtered Fact Table with 3-digit ISO codes:")
print(filtered_facttable_df.head())

In [ ]:
# Perform a left join on FLOW 
merged_df = filtered_facttable_df.merge(flow_df, left_on='FLOW', right_on='ID', how='left', suffixes=('', '_flow'))

# Perform a left join on TRANSPORT_MODE
merged_df = merged_df.merge(transport_mode_df, left_on='TRANSPORT_MODE', right_on='ID', how='left', suffixes=('', '_transport_mode'))

print("Filtered Fact Table Columns:")
print(merged_df.columns)

# Drop the redundant ID columns from flow and transport mode merges
merged_df = merged_df.drop(columns=[ 'FLOW', 'TRANSPORT_MODE', 'ID', 'ID_transport_mode'])


In [ ]:
# Ensure the key columns are of the same type
merged_df['DECLARANT'] = merged_df['DECLARANT'].astype(str)
merged_df['PARTNER'] = merged_df['PARTNER'].astype(str)

# Join with reporters_df to get declarant country names
merged_df = merged_df.merge(reporters_df[['ID', 'Name']], left_on='DECLARANT', right_on='ID', how='left', suffixes=('', '_declarant'))
merged_df = merged_df.rename(columns={'Name': 'DECLARANT_COUNTRY'})

# Join with partners_df to get partner country names
merged_df = merged_df.merge(partners_df[['ID', 'Name']], left_on='PARTNER', right_on='ID', how='left', suffixes=('', '_partner'))
merged_df = merged_df.rename(columns={'Name': 'PARTNER_COUNTRY'})

In [ ]:
# Drop the additional ID columns
merged_df = merged_df.drop(columns=['ID', 'ID_partner'])
print("Merged DataFrame:")
print(merged_df.head())

In [ ]:
# Define the desired column order
column_order = [
    'DECLARANT', 'DECLARANT_COUNTRY', 'DECLARANT_ISO_3', 'PARTNER', 'PARTNER_COUNTRY', 'PARTNER_ISO_3', 
    'TRADE_FLOW', 'TRANSPORTATION', 'PERIOD', 'PRODUCT_HS', 'HS4_NAME', 'VALUE_IN_EUROS', 'QUANTITY_IN_KG'
]

# Reorder the columns in the DataFrame
merged_df = merged_df[column_order]

print("Reordered DataFrame:")
print(merged_df.head())

In [ ]:
print("DataFrame Information:")
print(merged_df.info())

print("\nSummary Statistics:")
print(merged_df.describe(include='all'))

print("\nMissing Values:")
print(merged_df.isnull().sum())

print("\nData Types:")
print(merged_df.dtypes)

In [ ]:
# Find rows where both QUANTITY_IN_KG and VALUE_IN_EUROS are 0
zero_both_df = merged_df[(merged_df['QUANTITY_IN_KG'] == 0) & (merged_df['VALUE_IN_EUROS'] == 0)]

zero_both_count = zero_both_df.shape[0]

print(f"Total rows with both QUANTITY_IN_KG and VALUE_IN_EUROS equal to 0: {zero_both_count}")

# Find rows where VALUE_IN_EUROS is 0
zero_value_df = merged_df[merged_df['VALUE_IN_EUROS'] == 0]

print(zero_value_df.head())

zero_value_count = zero_value_df.shape[0]

print(f"Total rows with VALUE_IN_EUROS equal to 0: {zero_value_count}")

# Find rows where QUANTITY_IN_KG is 0
zero_quantity_df = merged_df[merged_df['QUANTITY_IN_KG'] == 0]

print(zero_quantity_df.head())

zero_quantity_count = zero_quantity_df.shape[0]

print(f"Total rows with QUANTITY_IN_KG equal to 0: {zero_quantity_count}")


In [ ]:
filtered_df=merged_df.copy()

# Display rows where PARTNER_ISO_3 is missing
missing_partner_iso_3 = filtered_df[filtered_df['PARTNER_ISO_3'].isna()]

print("Rows with missing PARTNER_ISO_3 values:")
print(missing_partner_iso_3.head())
print(f"Total missing values in PARTNER_ISO_3: {missing_partner_iso_3.shape[0]}")

In [ ]:
# Find distinct NAME_partner values
distinct_name_partner = missing_partner_iso_3['PARTNER_COUNTRY'].dropna().unique()

print(f"Total distinct NAME_partner values with missing PARTNER_ISO_3: {len(distinct_name_partner)}")
print("Distinct NAME_partner values with missing PARTNER_ISO_3:")
for name in distinct_name_partner:
    print(name)

In [ ]:
filtered_df['PARTNER_COUNTRY'] = filtered_df['PARTNER_COUNTRY'].str.strip()

# Additional cleaning for specific cases
filtered_df['PARTNER_COUNTRY'] = filtered_df['PARTNER_COUNTRY'].replace({
    'Countries and territories not specified within the framework of trade with third countries': 'Not specified (third countries)',
    'Stores and Provisions, Extra': 'Not specified (third countries)',
    'Stores and provisions within the framework of trade with third countries': 'Not specified (third countries)',
    'High seas': 'Not specified (third countries)',
    'Countries and territories not specified for commercial or military reasons in the framework of trade with third countries': 'Not specified (third countries)',
    
})

iso3_mapping = {
    'Liechtenstein': 'LIE',
    'Kosovo': 'XKX',
    'Montenegro': 'MNE',
    'Serbia': 'SRB',
    'Congo, Democratic Republic of': 'COD',
    'Congo,  Democratic Republic of': 'COD',
    'Curaçao': 'CUW',
    'Sint Maarten (Dutch part)': 'SXM',
    'South Sudan': 'SSD',
    'Namibia': 'NAM',
    'Bonaire, Sint Eustatius and Saba': 'BES',
    'Saint Barthélemy': 'BLM',
    'Timor-Leste': 'TLS',
    'Antarctica': 'ATA',
    'Guam': 'GUM',
    'Holy See (Vatican City State)': 'VAT',
    'American Samoa': 'ASM',
    'Ceuta': 'XC',
    'Melilla': 'ML',
    'Canary Islands': 'IC',
    'Virgin Islands, United States': 'VIR',
    'Not specified (third countries)': 'XXX'
}

# Update the PARTNER_ISO_3 column based on the iso3_mapping
for partner_name, iso3_code in iso3_mapping.items():
    filtered_df.loc[filtered_df['PARTNER_COUNTRY'] == partner_name, 'PARTNER_ISO_3'] = iso3_code

# Check which rows still have missing PARTNER_ISO_3
missing_partner_iso_3 = filtered_df[filtered_df['PARTNER_ISO_3'].isna()]
missing_count = missing_partner_iso_3.shape[0]
distinct_name_partner_missing = missing_partner_iso_3['PARTNER_COUNTRY'].dropna().unique()

print(f"Total rows with missing PARTNER_ISO_3: {missing_count}")
print("Distinct PARTNER_COUNTRY values with missing PARTNER_ISO_3:")
print(distinct_name_partner_missing)

# Final DataFrame
final_df = filtered_df.copy()

print("Final DataFrame:")
print(final_df.head())
print(f"Total rows in the final DataFrame: {final_df.shape[0]}")

In [ ]:
csv_file_path = '/Users/bercinersoz/Desktop/countries_codes_and_coordinates.csv'
excel_file_path = '/Users/bercinersoz/Desktop/final_data_with_coordinates.xlsx'

coordinates_df = pd.read_csv(csv_file_path)

# Remove double quotes in all cells
coordinates_df = coordinates_df.applymap(lambda x: x.replace('"', '') if isinstance(x, str) else x)

coordinates_df['Latitude (average)'] = pd.to_numeric(coordinates_df['Latitude (average)'], errors='coerce')
coordinates_df['Longitude (average)'] = pd.to_numeric(coordinates_df['Longitude (average)'], errors='coerce')

with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
    final_df.to_excel(writer, sheet_name='Final Data', index=False)
    melted_gdp_df.to_excel(writer, sheet_name='GDP Data', index=False)
    coordinates_df.to_excel(writer, sheet_name='Country Codes', index=False)

print(f"DataFrames have been written to {excel_file_path}")

In [ ]:
merged_df.head()

In [ ]:
# Filter the dataframe for Spain as the declarant and for the year 2020
spain_2020_df = final_df[(final_df['DECLARANT_ISO_3'] == 'ESP') & (final_df['PERIOD'] == '2020') & (final_df['TRADE_FLOW'] == 'Export')]

print(spain_2020_df.head())


# Calculate the total Value in Euro for Spain in 2020
total_value_euro_spain_2020 = spain_2020_df['VALUE_IN_EUROS'].sum()

# Display the total Value in Euro for Spain in 2020
print(f"Total Value in Euro for Spain in 2020: {total_value_euro_spain_2020}")